In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.models import KnownModelName
from pastel.classifiers import classify_images, classify_insurance_text, classify_insurance_image
from pastel.parsers import parse_assertion, parse_evidence, check_grammar
from pprint import pprint
import pandas as pd
import logfire
import os
from pastel.helpers import load_images_from_directory, create_consolidated_image
from pastel.models import (
    InsightModel,
    InsightValidation,
    InsightPlots,
    InputModel,
    PremiseValidation,
    BaseImage,
)
from pastel.prompts import EVALUATION_PROMPT, CLAIMS_VALIDATION_PROMPT

from IPython.display import display, Markdown

logfire.configure(token=os.getenv("PYDANTIC_LOGFIRE_TOKEN"))
logfire.instrument_openai()
logfire.instrument_anthropic()

OPENAI_MODEL: KnownModelName = "openai:gpt-4o-mini"
ANTHROPIC_MODEL: KnownModelName = "anthropic:claude-3-5-sonnet-latest"

index = 8

In [3]:
insights = pd.read_excel("../data/Insights.xlsx")

input_model = InputModel(
    name=insights["programname"][index],
    insight=insights["insight"][index],
    line_of_business=insights["line_of_business"][index],
)

assertion = await parse_assertion(input_model)
insight = await parse_evidence(assertion)

08:02:04.651 Chat Completion with 'gpt-4o-mini' [LLM]


Logfire project URL: ]8;id=705989;https://logfire.pydantic.dev/xmandeng/pastel\https://logfire.pydantic.dev/xmandeng/pastel]8;;\

08:02:10.686 Chat Completion with 'gpt-4o-mini' [LLM]


In [4]:
# Retrieve data sets
lrs = pd.read_excel("../data/lrs.xlsx")
lrs_data = lrs.loc[lrs["programname"] == insight.name].drop(columns=["programname"])

In [5]:
insight.name

'vbOJFqJ0BOzEXIi9uBzgow+z55EcdgzYq6ntebANax8='

In [6]:
images = InsightPlots(plots=load_images_from_directory(f"../data/{insight.name.replace('/', '-')}"))
images

InsightPlots(plots={'severity.png': BaseImage(image_path=PosixPath('../data/vbOJFqJ0BOzEXIi9uBzgow+z55EcdgzYq6ntebANax8=/severity.png'), image=<PIL.Image.Image image mode=RGB size=458x344 at 0x123AC8690>), 'pricing.png': BaseImage(image_path=PosixPath('../data/vbOJFqJ0BOzEXIi9uBzgow+z55EcdgzYq6ntebANax8=/pricing.png'), image=<PIL.Image.Image image mode=RGB size=426x262 at 0x12350C710>), 'frequency.png': BaseImage(image_path=PosixPath('../data/vbOJFqJ0BOzEXIi9uBzgow+z55EcdgzYq6ntebANax8=/frequency.png'), image=<PIL.Image.Image image mode=RGB size=451x311 at 0x123520690>), 'cut2.png': BaseImage(image_path=PosixPath('../data/vbOJFqJ0BOzEXIi9uBzgow+z55EcdgzYq6ntebANax8=/cut2.png'), image=<PIL.Image.Image image mode=RGB size=426x213 at 0x123520A10>), 'cut3.png': BaseImage(image_path=PosixPath('../data/vbOJFqJ0BOzEXIi9uBzgow+z55EcdgzYq6ntebANax8=/cut3.png'), image=<PIL.Image.Image image mode=RGB size=426x213 at 0x123523750>), 'cut1.png': BaseImage(image_path=PosixPath('../data/vbOJFqJ0BOzEXI

In [7]:
# Example usage
consolidated_image = create_consolidated_image(lrs_data, images, insight)

In [8]:
consolidated_image

BaseImage(image_path=PosixPath('../data/vbOJFqJ0BOzEXIi9uBzgow+z55EcdgzYq6ntebANax8=_consolidated_image.png'), image=<PIL.Image.Image image mode=RGB size=1200x2000 at 0x123AE24D0>)

In [9]:
premises = await classify_insurance_image(consolidated_image, insight.evidence)

08:02:12.587 Message with 'claude-3-5-sonnet-latest' [LLM]


In [10]:
premises

[PremiseValidation(claim='The policies written in FL appear to have higher loss ratios than other states', status='True', confidence='High', reasoning='Looking at the bottom chart showing loss ratios by state, Florida (FL) shows a loss ratio of approximately 0.32-0.33, which is indeed higher than most other states shown (NY, Other, CO, AZ). This is clearly visible in the data and supports the statement.'),
 PremiseValidation(claim='There is a considerable increase in price in the most recent months', status='False', confidence='High', reasoning="The 'Written Premium per Risk and Days Covered' graph (pricing.png) shows relatively flat pricing trends over time, with only a very slight upward movement towards the end. This minor change cannot be characterized as a 'considerable increase' - the line remains largely stable throughout the shown time period with minimal variation.")]